In [1]:
import pandas as pd


In [39]:
results_df = pd.read_csv('../data/States_df.csv')

In [40]:
results_df.head(2)

,District,State,cid,FirstLastP,Rcpts,Spent,PACs,Indivs,Cand,Other,...,Result,CRPICO,IncCID,Incumbent,primarydate,DistIDCurr,capeye,sort,SmLgIndivsNote,Party
0,1,Alabama,N00044245,Jerry Carl (R),1971321.50,1859348.91,387000.0,1044195.95,434655.5,105470.05,...,W,O,NaN,NaN,2020-03-03 00:00:00 +0000,,0,2,N,"['Jerry', ' ', 'Carl', ' ', '(R)']"
1,1,Alabama,N00044750,James Averhart (D),80094.95,78973.24,0.0,50849.95,29245.0,0.00,...,L,O,NaN,NaN,2020-03-03 00:00:00 +0000,,0,2,N,"['James', ' ', 'Averhart', ' ', '(D)']"


In [41]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264 entries, 0 to 1263
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   District        1264 non-null   int64  
 1   State           1264 non-null   object 
 2   cid             1264 non-null   object 
 3   FirstLastP      1264 non-null   object 
 4   Rcpts           1264 non-null   float64
 5   Spent           1264 non-null   float64
 6   PACs            1264 non-null   float64
 7   Indivs          1264 non-null   float64
 8   Cand            1264 non-null   float64
 9   Other           1264 non-null   float64
 10  EndCash         1264 non-null   float64
 11  LgIndivs        1264 non-null   float64
 12  SmIndivs        1264 non-null   float64
 13  enddate         901 non-null    object 
 14  ReleaseDate     1264 non-null   object 
 15  VotePercent     1264 non-null   float64
 16  Result          1264 non-null   object 
 17  CRPICO          1264 non-null   o

- how often does the candidate who raised the most money win? 
- group by state and district -- added new column so only need to group by the one column
- do you put in a flag to say this person raised the most, already have the win in the results column 
- then see out of x districts and states how many of them (so count total races) and then count how many people won and divide that by the total seats


In [42]:
# how many unique districts and states there are 
# making new column in df to combine district and state
results_df['st_dist'] = results_df['State'] + results_df['District'].astype(str)
total_seats = results_df['st_dist'].nunique()
total_seats


435

In [ ]:
# flagging row for who raised the most for each state district, function created below to do this, this was code to see group by working
results_df.groupby('st_dist')['Rcpts'].max()


st_dist
Alabama1      1971321.50
Alabama2       650806.75
Alabama3      1193111.43
Alabama4      1255076.11
Alabama5       655364.80
                 ...    
Wisconsin5    1155720.78
Wisconsin6    1815756.42
Wisconsin7    2637459.08
Wisconsin8    3202904.74
Wyoming1      3003883.34
Name: Rcpts, Length: 435, dtype: float64

In [ ]:
# function to determine who raised the most in the group 
def max_cont_dist(df, dist, raised):
    highest_raised = df.groupby(dist)[raised].transform('max')
    return (df[raised] == highest_raised).astype(int)

# adding column to flag the row that raised the most per group
results_df['max_raised'] = max_cont_dist(results_df, 'st_dist', 'Rcpts')

In [61]:
# counting how many groups where the person that won raised the most money
winners = results_df.groupby('st_dist')[['Result', 'max_raised']].apply(lambda x: (x['Result'] == 'W') & (x['max_raised'] == 1)).sum()
winners

384

In [63]:
# percentage of times where the most money raised wins the race
(winners / total_seats) * 100

88.27586206896552

88.27% of the time the person who raised the most money won the campaign